In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy.random as nr
import sklearn.model_selection as ms
import math


%matplotlib inline

In [3]:
data = pd.read_json('https://raw.githubusercontent.com/wncc/learners-space/master/Machine%20Learning/MId%20Term%20Assignment/data.json')
data.head()

,0,1,2
0,-3.005382,-1.701191,1
1,-2.908105,-1.699589,1
2,-2.958954,-1.660940,1
3,-2.922840,-1.638722,1
4,-2.924240,-1.508256,1


In [4]:
label_counts = data.iloc[:,2].value_counts()
print(label_counts)

1    4679
0    1321
Name: 2, dtype: int64


In [5]:
# labels = np.array(data.iloc[:, 2])

In [6]:
nr.seed(3432)

X = data.iloc[:,:2]
Y = data.iloc[:,2:]
x_train,x_test,y_train,y_test = ms.train_test_split(X,Y, test_size = 0.2)

In [7]:
np.array(x_train).reshape(4800,2)
np.array(y_train).reshape(4800,1)



array([[0],
       [1],
       [0],
       ...,
       [1],
       [0],
       [0]])

In [8]:
print(x_train)

             0         1
2713 -0.371902 -1.251144
695  -2.461700  1.556300
2473 -0.643855  0.760351
5244  2.137680 -0.110429
3775  0.690295  0.829854
...        ...       ...
5172  2.047902  0.714714
4692  1.639363  1.388698
45   -2.993520 -0.144795
2311 -0.799276 -1.220207
3185  0.018570  1.196394

[4800 rows x 2 columns]


In [9]:
# Function  
def f(A, coef):
  fun = np.dot(A,coef.reshape(5,1))
  return fun

In [10]:
# Hypothesis
def h(A, coef):
  return np.array(1/(1+(np.exp(-f(A,coef)))) )

In [11]:
# Loss function
def l(y, A, coef):   # y shouldn't be the predicted values
  return (-np.dot(y.T, np.log(h(A, coef))) - np.dot((1-y).T ,np.log(1-h(A, coef))) )/y.shape[0]
  # this function will give us the mean loss 

In [12]:
# Gradient 
def g(A, coef, y):
  return np.dot(np.array(h(A, coef).reshape(y.shape[0],1) - y).reshape([1,y.shape[0]]), A)/y.shape[0]
  # here we will get a 1x5 matrix consisting of the gradient wrt each of the coeff

In [13]:
# Gradient Descent 
def gradesc(A, y, coef,L):       # here coef is column vector 
  for i in range (250):
    coef = coef.reshape(5,1) - L*g(A, coef, y).reshape(5,1)
  return coef  


In [14]:

A0 = np.array(x_train[0]**4).reshape(1, x_train.shape[0])
A1 = np.array(x_train[0]**3 * x_train[1]).reshape(1, x_train.shape[0])
A2 = np.array(x_train[0]**2 * x_train[1]**2).reshape(1, x_train.shape[0])
A3 = np.array(x_train[0]**1 * x_train[1]**3).reshape(1, x_train.shape[0])
A4 = np.array(x_train[1]**4 ).reshape(1, x_train.shape[0])
A = np.vstack((A0, A1, A2, A3, A4))
A = A.T
print(A)
A.shape

[[ 1.91298605e-02  6.43563036e-02  2.16506221e-01  7.28366007e-01
   2.45035471e+00]
 [ 3.67232118e+01 -2.32166122e+01  1.46776672e+01 -9.27930017e+00
   5.86642350e+00]
 [ 1.71850800e-01 -2.02944770e-01  2.39664753e-01 -2.83028698e-01
   3.34238735e-01]
 ...
 [ 8.03024531e+01  3.88419512e+00  1.87876847e-01  9.08752230e-03
   4.39559545e-04]
 [ 4.08119830e-01  6.23052177e-01  9.51176558e-01  1.45210446e+00
   2.21684118e+00]
 [ 1.18913640e-07  7.66122030e-06  4.93587586e-04  3.18002480e-02
   2.04878688e+00]]


(4800, 5)

In [15]:
coef = np.array([0.001, 0.001, 0.001, 0.001, 0.001])
type(coef)
coef.shape

(5,)

In [16]:
gradesc(A,y_train, coef, 0.001)

array([[ 0.19800197],
       [-0.0008506 ],
       [ 0.040981  ],
       [-0.00106869],
       [-0.03651595]])

In [17]:
def predict(x_test, coef = gradesc(A,y_train, coef, 0.001) ):
  A0 = np.array(x_test[0]**4).reshape(1, x_test.shape[0])
  A1 = np.array(x_test[0]**3 * x_test[1]).reshape(1, x_test.shape[0])
  A2 = np.array(x_test[0]**2 * x_test[1]**2).reshape(1, x_test.shape[0])
  A3 = np.array(x_test[0]**1 * x_test[1]**3).reshape(1, x_test.shape[0])
  A4 = np.array(x_test[1]**4 ).reshape(1, x_test.shape[0])
  A_ = np.vstack((A0, A1, A2, A3, A4))
  A_ = A_.T
  print('Mean Loss :' ,np.sum(l(y_test,A_,coef)))
  # print(A_)
  return h(A_, coef)

In [18]:
probs = predict(x_test)
print(probs)

Mean Loss : 0.32309249865730216
[[0.50786053]
 [0.50155393]
 [0.50329794]
 ...
 [0.99042167]
 [0.72070841]
 [0.62494952]]


In [34]:
def score_model(probs, threshold):
    return np.array([1 if x > threshold else 0 for x in probs[:]])
y_pred = score_model(probs, 0.5)
pred_df = pd.DataFrame(y_pred)
print('Predicted values')
print(pred_df.value_counts())

Predicted values
1    1035
0     165
dtype: int64


In [31]:
print('Actual values')
y_test.value_counts()

Actual values


2
1    974
0    226
dtype: int64

In [36]:
from sklearn import preprocessing
import sklearn.model_selection as ms
from sklearn import linear_model
import sklearn.metrics as sklm

def print_metrics(labels, scores):
    metrics = sklm.precision_recall_fscore_support(labels, scores)
    conf = sklm.confusion_matrix(labels, scores)
    print('                 Confusion matrix')
    print('                 Score positive    Score negative')
    print('Actual positive    %6d' % conf[1,1] + '             %5d' % conf[1,0])
    print('Actual negative    %6d' % conf[0,1] + '             %5d' % conf[0,0])
    print('')
    print('Accuracy  %0.2f' % sklm.accuracy_score(labels, scores))
    print(' ')
    print('           Positive      Negative')
    print('Num case   %6d' % metrics[3][1] + '        %6d' % metrics[3][0])
    print('Precision  %6.2f' % metrics[0][1] + '        %6.2f' % metrics[0][0])
    print('Recall     %6.2f' % metrics[1][1] + '        %6.2f' % metrics[1][0])
    print('F1         %6.2f' % metrics[2][1] + '        %6.2f' % metrics[2][0])


    
print_metrics(y_test, y_pred)

                 Confusion matrix
                 Score positive    Score negative
Actual positive       966                 8
Actual negative        69               157

Accuracy  0.94
 
           Positive      Negative
Num case      974           226
Precision    0.93          0.95
Recall       0.99          0.69
F1           0.96          0.80


Here we can see that the accuracy is high ... though accuracy is not the best metric to evaluate ... the Precision and Recall are also good enough.